In [1]:
import os.path
import re
import html
import string

In [40]:
documents = []
source_folder = os.path.join('.', 'reuters21578')   # join two strings with path specifier independent from the OS


# get every article as a document where the id of the document is NEWID parameter of the article
def getDocuments():


    for file in os.listdir(source_folder):
        
        # get all files with an extension .sgm
        if file.endswith('.sgm'):
            with open(os.path.join(source_folder, file), 'r') as f:
                file_as_string = f.read()

            # while splitting based on </REUTERS> tag, we will have (# of articles) + 1 elements in the list named reuters.
            # the last element in the list does not contain any information because regular expression finds the closing tag
            # for an article and splits from that point, resulting in that the last element is below the last article.
            reuters = re.split('</REUTERS>', file_as_string)

            # traverse articles one by one
            for reuter in reuters[:-1]:

                ### get the docid
                # find the pattern NEWID="[number]"> and get the number from the string.
                # then, convert the string to an integer
                doc_id_search = re.search(r"NEWID=\"([0-9]+)\">", reuter)
                doc_id = int(doc_id_search.group(1))
                

                # get rid of html escape characters like '&lt;', '&#3;'
                reuter = html.unescape(reuter)
                doc = ""

                ### get the title
                # find the pattern "<TEXT...<TITLE>[title_text]</TITLE>..." and get the title_text from the string
                title_search = re.search(r'<TEXT(.|\n)*<TITLE>((.|\n)*)</TITLE', reuter)
                if title_search is not None: # if title is found
                    title = title_search.group(2)
                    doc += title.lower()

                ### get the body
                # find the pattern "<TEXT...<BODY>[body_text]</BODY>..." and get the body_text from the string
                body_search = re.search(r'<TEXT(.|\n)*<BODY>((.|\n)*)</BODY>', reuter)
                if body_search is not None: # if the body is found
                    body = body_search.group(2)
                    doc += " " + body[0].lower() + body[1:]

                # if both title and body cannot be found, that means that the article is in UNPROC format and it contains
                # only <TEXT> parameter, not <TITLE> or <BODY>
                if (title_search is None) and (body_search is None):
                    try:
                        ### get the text
                        # find the pattern "<TEXT...[body_text]</TEXT>" and get the body_text from the string
                        text_search = re.search(r'<TEXT.+\n((.|\n)+)</TEXT>', reuter)
                        body = text_search.group(1)
                        doc = body[0].lower() + body[1:]
                    except:
                        print(reuter, '\nerror')
                        return

                ### get rid of "reuter" at the end
                # if the document contains "Reuter" or "REUTER" with some whitespace characters at the end, cut that part away from the document 
                # reuter_finish_search = re.search('((.|\n)+)\s*\Z', doc)
                # doc = reuter_finish_search.group(1) if reuter_finish_search is not None else doc

                # gather the documents in an array
                documents.append([doc_id, doc])

            f.close()


getDocuments()

In [41]:
def getClitics():

    clitics = set()
    clitic_path = os.path.join('.', 'clitics.txt')

    with open(clitic_path) as stop_file:
        stop_lines = stop_file.readlines()

    for line in stop_lines:
        stopword = line[:-1]
        clitics.add(stopword)

    return clitics

In [42]:
def normalizeDocuments():

    # to hold unique tokens(terms), I decided to use set data structure so that there will not be duplicate tokens at the end
    # one for terms before the normalization(case-folding and hyphen detection) process and one for after the normalization process 
    dictionary_before_normalization = set()
    dictionary_after_normalization = set()

    # number of tokens before and after the normalization process
    tokens_before_normalization = 0
    tokens_after_normalization = 0


    for document_pair_index in range(len(documents)):

        # get the document as a string
        document = documents[document_pair_index][1]

        # simple splitting with respect to whitespaces
        word_list = re.split(r'\s+', document)
        tokens_before_normalization += len(word_list)

        # get clitics: we're, i'm, etc.
        clitics = getClitics()

        # regular expression that will detect either a word, words with hyphens included, or a token composing of numbers
        # with special characters('/', ':', '.', ',') like 16/20, 19.02.2021, etc.
        shave_string = f'[{string.punctuation}]*([\d/:.,]+)|([\w-]+)[{string.punctuation}]*'

        # a sentence will probably end with one of the following punctuation marks: '.', '!', '?', '...'
        end_of_sentence = r'[.?!]|[...]'
        # a flag that will show that the previous word is the last word of the sentence, which means the current word is the beginning of the sentence
        sentence_beginning = False

        # final document as a list of its tokens
        final_document = []

        for word in word_list:
            
            if word.lower() not in clitics:

                # shave the token
                shaved = re.search(shave_string, word)
                if shaved is not None:
                    # as described in the definition of shave_string, token can be in one of the 2 different forms
                    token = shaved.group(1) if shaved.group(1) is not None else shaved.group(2)

                    # keep terms before the normalization
                    dictionary_before_normalization.add(token)

                    # detect hyphenated words
                    dash_search = re.search('-', token)

                    # after this conditional block, we will have a list of tokens named as token_splitted
                    if dash_search:
                        # if the first character of the hyphenated word is upper, like Hewett-Pickard or New York-San Fransisco, then they should be 
                        # splitted and taken as different strings. if the first character is lower, then all hyphens should be deleted and the result will be
                        # one word('know-how' to 'knowhow')
                        tokens_splitted = re.split('-', token) if token[0].isupper() else [re.sub('-', '', token)]
                    else:
                        # if no hyphen detected, then the token is taken directly
                        tokens_splitted = [token]

                    for splitted in tokens_splitted:
                        if splitted:
                            # if the first character of the token is upper and it is not the beginning of the sentence, then keep the token with uppercase letters.
                            # otherwise, lower the letters
                            splitted = splitted if (splitted[0].isupper() and (not sentence_beginning)) else splitted.lower()
                            
                            # keep terms after normalization
                            dictionary_after_normalization.add(splitted)
                            # keep the number of tokens after normalization
                            tokens_after_normalization += 1

                            final_document.append(splitted)

                end_of_sentence_search = re.search(end_of_sentence, word)
                sentence_beginning = end_of_sentence_search is not None
            else:
                # keep terms before the normalization
                dictionary_before_normalization.add(word)

                token = word.lower()

                # keep terms after normalization
                dictionary_after_normalization.add(word)
                # keep the number of tokens after normalization
                tokens_after_normalization += 1
                
                final_document.append(splitted)

        documents[document_pair_index][1] = final_document

    
    return (dictionary_after_normalization, len(dictionary_before_normalization), tokens_before_normalization, tokens_after_normalization)


In [43]:
(dictionary, len_dictionary_before_normalization, tokens_before_normalization, tokens_after_normalization) = normalizeDocuments()

In [44]:
print(len(dictionary), len_dictionary_before_normalization, tokens_before_normalization, tokens_after_normalization)
print(documents[0][1])

86258 88660 2781148 2760861
['bahia', 'cocoa', 'review', 'showers', 'continued', 'throughout', 'the', 'week', 'in', 'the', 'Bahia', 'cocoa', 'zone', 'alleviating', 'the', 'drought', 'since', 'early', 'January', 'and', 'improving', 'prospects', 'for', 'the', 'coming', 'temporao', 'although', 'normal', 'humidity', 'levels', 'have', 'not', 'been', 'restored', 'Comissaria', 'Smith', 'said', 'in', 'its', 'weekly', 'review', 'the', 'dry', 'period', 'means', 'the', 'temporao', 'will', 'be', 'late', 'this', 'year', 'arrivals', 'for', 'the', 'week', 'ended', 'February', '22', 'were', '155,221', 'bags', 'of', '60', 'kilos', 'making', 'a', 'cumulative', 'total', 'for', 'the', 'season', 'of', '5.93', 'mln', 'against', '5.81', 'at', 'the', 'same', 'stage', 'last', 'year', 'again', 'it', 'seems', 'that', 'cocoa', 'delivered', 'earlier', 'on', 'consignment', 'was', 'included', 'in', 'the', 'arrivals', 'figures', 'comissaria', 'Smith', 'said', 'there', 'is', 'still', 'some', 'doubt', 'as', 'to', 'how'